In [1]:
import pybamm;import pandas as pd   ;import numpy as np;import os;import matplotlib.pyplot as plt;import os;#import imageio
from scipy.io import savemat,loadmat;from pybamm import constants,exp;
import matplotlib as mpl; 
fs=17; # or we can set import matplotlib.pyplot as plt then say 'mpl.rc...'
import openpyxl
import traceback
import multiprocessing
font = {'family' : 'DejaVu Sans','size'   : fs}
mpl.rc('font', **font)

In [4]:
V_max = 4.2;        V_min = 2.5;
exp  = pybamm.Experiment(
    [ (
        f"Discharge at 1 C until {V_min} V", 
        f"Charge at 1 C until {V_max} V", 
        f"Hold at {V_max} V until C/5")    ] * 1 )  
Model_0  = pybamm.lithium_ion.DFN(options={
    "solvent diffusion": "double spatial consume w refill",
    "electrolyte conductivity": "full"  ,
    }) 
Model_1  = pybamm.lithium_ion.DFN(options={
    "solvent diffusion": "double spatial consume w refill",
    "electrolyte conductivity": "sol full"  ,
    }) 

In [6]:
# Run model without electrolyte dry out
Para_0=pybamm.ParameterValues("Li2023_ECdrag")

In [7]:
Sim_0    = pybamm.Simulation(
    Model_0, experiment = exp,
    parameter_values = Para_0,
    solver = pybamm.CasadiSolver(),)       
Sol_0    = Sim_0.solve()

In [8]:
Sim_1    = pybamm.Simulation(
    Model_1, experiment = exp,
    parameter_values = Para_0,
    solver = pybamm.CasadiSolver(),)       
Sol_1    = Sim_1.solve()

In [10]:
label = ["Sol NOT affect potential","Sol affect potential",] 
output_variables3 = [
    "Electrolyte potential [V]",
    "Positive electrode potential [V]",
    "Negative electrode potential [V]",
    "Positive current collector potential [V]", 
    "Terminal voltage [V]", 
]
quick_plot = pybamm.QuickPlot(
    [Sol_0,Sol_1], output_variables3,label,variable_limits='fixed',
    time_unit='hours',
    spatial_unit='mm',     #  (“m”, “mm”, or “um”)
    n_rows=2) #figsize = (18,12),
quick_plot.dynamic_plot()

interactive(children=(FloatSlider(value=0.0, description='t', max=2.1787232685683717, step=0.02178723268568371…

In [14]:
quick_plot = pybamm.QuickPlot(Sol_0) #     
quick_plot.dynamic_plot();


interactive(children=(FloatSlider(value=0.0, description='t', max=4.134631245347678, step=0.04134631245347678)…

In [15]:
"""Check that the model is not under- or over-determined."""
# Equations (differential and algebraic)
# Get all the variables from differential and algebraic equations
vars_in_rhs_keys = set()
vars_in_algebraic_keys = set()
vars_in_eqns = set()
# Get all variables ids from rhs and algebraic keys and equations, and
# from boundary conditions
# For equations we look through the whole expression tree.
# "Variables" can be Concatenations so we also have to look in the whole
# expression tree
unpacker = pybamm.SymbolUnpacker((pybamm.Variable, pybamm.VariableDot))

for var, eqn in Model_0.rhs.items():
    # Find all variables and variabledot objects
    vars_in_rhs_keys_dict = unpacker.unpack_symbol(var)
    vars_in_eqns_dict = unpacker.unpack_symbol(eqn)

    # Store ids only
    # Look only for Variable (not VariableDot) in rhs keys
    vars_in_rhs_keys.update(
        [
            var_id
            for var_id, var in vars_in_rhs_keys_dict.items()
            if isinstance(var, pybamm.Variable)
        ]
    )
    vars_in_eqns.update(vars_in_eqns_dict.keys())
for var, eqn in Model_0.algebraic.items():
    # Find all variables and variabledot objects
    vars_in_algebraic_keys_dict = unpacker.unpack_symbol(var)
    vars_in_eqns_dict = unpacker.unpack_symbol(eqn)

    # Store ids only
    # Look only for Variable (not VariableDot) in algebraic keys
    vars_in_algebraic_keys.update(
        [
            var_id
            for var_id, var in vars_in_algebraic_keys_dict.items()
            if isinstance(var, pybamm.Variable)
        ]
    )
    vars_in_eqns.update(vars_in_eqns_dict.keys())
for var, side_eqn in Model_0.boundary_conditions.items():
    for side, (eqn, typ) in side_eqn.items():
        vars_in_eqns_dict = unpacker.unpack_symbol(eqn)
        vars_in_eqns.update(vars_in_eqns_dict.keys())

# If any keys are repeated between rhs and algebraic then the model is
# overdetermined
if not set(vars_in_rhs_keys).isdisjoint(vars_in_algebraic_keys):
    print("model is overdetermined (repeated keys)")
# If any algebraic keys don't appear in the eqns (or bcs) then the model is
# overdetermined (but rhs keys can be absent from the eqns, e.g. dcdt = -1 is
# fine)
# Skip this step after discretisation, as any variables in the equations will
# have been discretised to slices but keys will still be variables
extra_algebraic_keys = vars_in_algebraic_keys.difference(vars_in_eqns)
# If any variables in the equations don't appear in the keys then the model is
# underdetermined
vars_in_keys = vars_in_rhs_keys.union(vars_in_algebraic_keys)
extra_variables_in_equations = vars_in_eqns.difference(vars_in_keys)

# get ids of external variables
external_ids = {var.id for var in Model_0.external_variables}
for var in Model_0.external_variables:
    if isinstance(var, pybamm.Concatenation):
        child_ids = {child.id for child in var.children}
        external_ids = external_ids.union(child_ids)

extra_variables = extra_variables_in_equations.difference(external_ids)

if extra_variables:
    print("model is underdetermined (too many variables)")


AttributeError: 'set' object has no attribute 'items'

In [ ]:
for var, eqn in Model_0.rhs.items():
    # Find all variables and variabledot objects
    vars_in_rhs_keys_dict = unpacker.unpack_symbol(var)
    vars_in_eqns_dict = unpacker.unpack_symbol(eqn)
    print(vars_in_eqns_dict)
    print(vars_in_rhs_keys_dict)
    print('z')

In [ ]:
for var in Model_0.external_variables:
    print(var)

In [ ]:
print(Model_0.variable_names)
for key in Model_0._variables.keys():
    print(key)

In [ ]:
#print(Model_0.rhs.items())
for var, eqn in Model_0.rhs.items():
    print(var)
    #print(eqn)


In [ ]:
for var, eqn in Model_0.algebraic.items():
    print(var)

In [ ]:
print(extra_variables_in_equations)
print(extra_variables)
print(external_ids)

In [ ]:
Model_0.submodels